In [ ]:
!pip install /kaggle/input/uspto-whoosh-reloaded-2-7-5-patched/Whoosh_Reloaded-2.7.5-py2.py3-none-any.whl
import random
import whoosh
import pyarrow.parquet as pq
from collections import Counter
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import seaborn as sns
import squarify
import numpy as np
import os, os.path
from whoosh import index
from whoosh import qparser
from whoosh.fields import Schema, TEXT, KEYWORD, ID, STORED
from whoosh.analysis import StemmingAnalyzer
from whoosh.qparser import QueryParser
from whoosh.index import create_in, open_dir
import re


In [ ]:
pd.set_option('display.max_colwidth', None)
tot_df =pd.read_csv("/kaggle/input/cpctot/cpctot.dat", encoding='latin1')
cpc_tot_df=tot_df.fillna(0)

cpc_tot_df

In [ ]:
logical_min_results= (cpc_tot_df.min_class == cpc_tot_df.cpc_code)
logical_min_results

In [ ]:
cpctot_min_Logical_df=cpc_tot_df[logical_min_results ]  
cpctot_min_df=cpctot_min_Logical_df [['min_class','cpc_desc']]
cpctot_min_df

In [ ]:
cpctot_min_class_df=cpctot_min_df.rename (columns = {'cpc_desc': 'min_desc'})
cpctot_min_desc_df=cpctot_min_class_df.rename (columns = {'cpc_desc': 'min_desc'})
cpctot_min_class_df

In [ ]:
cpc_tot_df.set_index('min_class', inplace=True)
cpctot_min_desc_df.set_index('min_class', inplace=True)
cpc_tot_df.update(cpctot_min_desc_df)
cpc_tot_df_reset=cpc_tot_df.reset_index(drop = True)
cpc_tot_df_reset

In [ ]:
logical_sub_results= (cpc_tot_df_reset.sub_class == cpc_tot_df_reset.cpc_code)
#logical_sub_results_reset = logical_sub_results.reset_index(drop=True)
logical_sub_results

In [ ]:
cpctot_sub_Logical_df=cpc_tot_df_reset[logical_sub_results ]  
cpctot_sub_df=cpctot_sub_Logical_df [['sub_class','cpc_desc']]
cpctot_sub_df_reset = cpctot_sub_df.reset_index(drop=True)
cpctot_sub_df_reset

In [ ]:
cpctot_sub_class_df=cpctot_sub_df_reset.rename (columns = {'cpc_desc': 'sub_desc'})
#cpc_A_sub_desc_df=cpctot_sub_class_df.rename (columns = {'cpc_desc': 'sub_desc'})
cpctot_sub_class_df

In [ ]:
cpc_tot_df_reset.set_index('sub_class', inplace=True)
cpctot_sub_class_df.set_index('sub_class', inplace=True)
cpc_tot_df_reset.update(cpctot_sub_class_df)
cpc_tot_df_reset_reset=cpc_tot_df_reset.reset_index(drop = True)
cpc_tot_df_reset_reset

In [ ]:
logical_fin_results= (cpc_tot_df_reset_reset.fin_class == cpc_tot_df_reset_reset.cpc_code)
#logical_fin_results= ((cpc_tot_df_reset_reset.fin_class == cpc_tot_df_reset_reset.cpc_code.str[:5]) & (cpc_tot_df_reset_reset.cpc_code.str[-2:-1] == 0))
logical_fin_results

In [ ]:
cpctot_fin_Logical_df=cpc_tot_df_reset_reset[logical_fin_results ]  
cpctot_fin_df=cpctot_fin_Logical_df [['fin_class','cpc_desc']]
cpctot_fin_df

In [ ]:
cpctot_fin_class_df=cpctot_fin_df.rename (columns = {'cpc_desc': 'fin_desc'})
cpc_A_fin_desc_df=cpctot_fin_class_df.rename (columns = {'cpc_desc': 'fin_desc'})
cpctot_fin_class_df

In [ ]:
cpc_tot_df_reset_reset.set_index('fin_class', inplace=True)
cpc_tot_df_reset_reset.update(cpctot_fin_class_df.set_index('fin_class'))
cpc_tot_df_reset_reset_reset = cpc_tot_df_reset_reset.reset_index()
cpc_tot_df_reset_reset_reset

In [ ]:
cpctot_fin_Logical_df=cpc_tot_df_reset_reset_reset[(cpc_tot_df_reset_reset_reset['cpc_code'].str.find('/')>4) & (cpc_tot_df_reset_reset_reset['det_class']==0)]
cpctot_fin_Logical_df

In [ ]:
cpctot_fin_df=cpctot_fin_Logical_df [['fin_class','cpc_desc']]
cpctot_fin_df

In [ ]:
cpctot_fin_class_df=cpctot_fin_df.rename (columns = {'cpc_desc': 'fin_desc'})
cpctot_fin_class_df

In [ ]:
#cpc_tot_df_reset_reset_reset.set_index('fin_class', inplace=True)
cpctot_fin_class_df.set_index('fin_class', inplace=True)
cpc_tot_df_reset_reset_reset.update(cpctot_fin_class_df)
cpc_tot_df_reset_reset_reset_reset = cpc_tot_df_reset_reset_reset.reset_index()
del cpc_tot_df_reset_reset_reset_reset['det_desc']
cons_cpc_tot_df=cpc_tot_df_reset_reset_reset_reset[['cpc_code','cpc_desc']]
cons_cpc_tot_df

In [ ]:
cons_cpc_tot_df.to_csv('cons_cpc_tot.csv')

In [ ]:
file_path = "/kaggle/input/uspto-explainable-ai/patent_metadata.parquet"
metadata_df = pd.read_parquet(file_path)

In [ ]:
publications_df=metadata_df.reset_index()
exploded_codes_df=publications_df.explode('cpc_codes')
all_publications_df=exploded_codes_df.rename (columns = {'cpc_codes': 'cpc_code'})
all_publications_df

In [ ]:
all_publications_df.dropna(subset=["cpc_code"], inplace=True)
trunc_publications=all_publications_df.copy()
trunc_publications

In [ ]:
del trunc_publications['filing_date']
trunc_publications

In [ ]:
del trunc_publications['publication_date']

trunc_publications

In [ ]:
del trunc_publications['family_id']
trunc_publications

In [ ]:
del trunc_publications['index']
trunc_publications

In [ ]:
import pandas as pd

# Read the JSON file into a DataFrame
jdf = pd.read_json('/kaggle/input/uspto-explainable-ai/train_index_patent_ids.json')

# Assuming `train_df` is supposed to be your DataFrame, define it or read it from somewhere
train_df = pd.DataFrame()

# Assign values from jdf to the `publication_number` column of train_df
train_df['publication_number'] = jdf[:]

# Reset the index of train_df
train_df.reset_index(drop=True, inplace=True)

train_df


In [ ]:
test_df = pd.read_csv('/kaggle/input/uspto-explainable-ai/test.csv', header = None)

In [ ]:
test_df.reset_index(drop=True)
#testdf= newdf[51:]
test_data=test_df.reset_index(drop=True)
test_data_df=test_data.loc[1:,:0,]
test_data_df=test_data_df.rename(columns={0: 'publication_number'})
test_data_df

In [ ]:
#test_add=pd.merge(all_publications_df,test_data_df,on='publication_number')
total_df=pd.concat([train_df,test_data_df])
total_df.to_csv('traintest.csv')
total_df

In [ ]:
train_add=pd.merge(trunc_publications,train_df,on='publication_number')
train_add

In [ ]:
test_add=pd.merge(trunc_publications,test_data_df,on='publication_number')
test_add

In [ ]:
sorted_add=pd.merge(trunc_publications,total_df,on='publication_number')
sorted_add

In [ ]:
merged_train_df=pd.merge(train_add,cons_cpc_tot_df,on='cpc_code')
merged_train_df.to_csv('merged_train.csv')
merged_train_df

In [ ]:
merged_test_df=pd.merge(test_add,cons_cpc_tot_df,on='cpc_code')
merged_test_df.to_csv('merged_test.csv')
merged_test_df

In [ ]:
merged_total_df=pd.merge(sorted_add,cons_cpc_tot_df,on='cpc_code')
merged_total_df

In [ ]:
pd.set_option('display.max_colwidth', None)
ttnearest_df = pd.DataFrame()
ttnearest_df = pd.read_csv('/kaggle/input/ttnearest-condensed/ttnearest_consolidated.csv')
ttnearest_df

In [ ]:
corrected_df=pd.DataFrame()
corrected_df =   pd.read_csv('/kaggle/input/corrected-merge-traintest/corrected_merge_traintest.csv')
corrected_df

In [ ]:
last_sub_heading_code = corrected_df[corrected_df['cpc_code'].str.endswith('/00')]
last_sub_heading_code

In [ ]:
only_cpc_code = pd.merge(corrected_df, last_sub_heading_code, how='left', indicator=True)
only_cpc_code = only_cpc_code[only_cpc_code['_merge'] == 'left_only'].drop('_merge', axis=1)
only_cpc_code

In [ ]:
pd.set_option('display.max_colwidth', None)
only_cpc_code

In [ ]:
only_cpc_code = only_cpc_code.replace(',', ' ', regex=True)

In [ ]:
only_cpc_code

In [ ]:
pd.set_option('display.max_colwidth', None)
import pandas as pd
corrected_test_df=pd.DataFrame()
corrected_test_df =   pd.read_csv('/kaggle/input/corrected-square-test/corrected_square_test.csv')
corrected_test_df

In [ ]:
schema = Schema(publication_number=TEXT(stored=True),
                cpc_desc=KEYWORD(stored=True))

In [ ]:
import os
from whoosh import index

if not os.path.exists("cpcdescdir"):
    os.mkdir("cpcdescdir")

cpcdescidx = index.create_in("cpcdescdir", schema)

In [ ]:
#ix = index.open_dir("cpcdescdir")

with cpcdescidx.writer() as writer:                                      

   for i in corrected_test_df.index:                                         
       writer.update_document(publication_number = str(corrected_test_df.loc[i,"publication_number"]), 
                              cpc_desc = str(corrected_test_df.loc[i, "cpc_desc"])) 

In [ ]:
from whoosh import index
from whoosh.fields import Schema, TEXT, ID
from whoosh.qparser import QueryParser

# Define the schema for the index
schema = Schema(
    publication_number=TEXT(stored=True),
    cpc_desc=KEYWORD(stored=True)
)
pd.set_option('display.max_rows', 500)
# Path to the index directory
index_dir = "/kaggle/working/cpcdescdir"

# Open the index
ix = index.open_dir(index_dir)

tdf = pd.DataFrame(columns = ['publication_number', 'nearest_numbers'])
refdf = pd.DataFrame(columns = ['publication_number', 'cpc_desc'])
querydf = pd.DataFrame(columns = ['publication_number', 'query'])
dict_list = []
# Print the schema
print("Schema:")
for field_name, field_type in ix.schema.items():
    print(f"{field_name}: {field_type}")

# Create a searcher object
with ix.searcher() as searcher:
    # Define a query parser and parse a query
    query_parser = QueryParser("cpc_desc", ix.schema)
    query = query_parser.parse("*")  # This will match all documents

    # Search the index
    #results = searcher.search(query)
    results = searcher.search(query, limit=None)
    results_list = []
    pub_number_list = []
    cpc_desc_list = []
    seen_publication_numbers = set()
    
    # Print the results
    #print("\nIndex Contents:")
    for result in results:
        #print(dict(result))
        #print(result['publication_number']) 
        result_dict = dict(result)
        results_list.append(result_dict)
        pub_number = result_dict.get('publication_number')
        # Check if the publication number is already seen
        
        if pub_number not in seen_publication_numbers:
            #cpc_query = result_dict.get('cpc_desc')
            
            
            #seen_publication_numbers.add(cpc_query)
            #qdf1 = pd.DataFrame(seen_publication_numbers)
            seen_publication_numbers.add(pub_number)
            pub_number_list.append(result_dict.get('publication_number'))
            cpc_desc_list.append(result_dict.get('cpc_desc'))
            #qdf2 = result['publication_number']
            #querydf = pd.concat([querydf,qdf1 ], ignore_index=True)
            #print(seen_publication_numbers)                     
            #print(result_dict)
            #print(result['publication_number'])
            selection = (ttnearest_df.publication_number == result['publication_number'])
            filtered_df = ttnearest_df[selection].reset_index(drop=True)
            tdf = pd.concat([tdf, filtered_df], ignore_index=True)
            #print(ttnearest_df[selection])
            #row_dict = {'publication_number': result['publication_number'], 'nearest_numbers': 'ttnearest_df.nearest_numbers'}
            #new_row = {'publication_number':result['publication_number'], 'nearest_numbers': 'ttnearest_df.nearest_numbers' }
            #dict_list.append(row_dict)
            #tdf = tdf.append(new_row, ignore_index=True) 
            #tdf = pd.DataFrame.from_dict(dict_list)
            picking  = (corrected_test_df.publication_number == result['publication_number'])
            referred_df = corrected_test_df[picking].reset_index(drop=True)
            refdf = pd.concat([refdf, referred_df], ignore_index=True)
            #print(result['publication_number'])
results_df = pd.DataFrame(results_list)
unique_results_df = results_df.drop_duplicates(subset='publication_number', keep='first')
unique_results_df.rename(columns={'cpc_desc': 'query'}, inplace=True)
unique_results_df = unique_results_df[[ 'publication_number','query']]
#tdf.to_csv('final_test_results.csv')

#seen_publication_numbers

unique_results_df.reset_index(drop=True)


In [ ]:
results_list=[]
qp = QueryParser("content", schema=cpcdescidx.schema)
for index, row in unique_results_df.iterrows():
    query_str = row['query']
    q = qp.parse(query_str)
# Perform the search

    with cpcdescidx.searcher() as s:
        results = s.search(q)
        result_string = str(results)
        if isinstance(result_string, str):
            terms_pattern = re.compile(r"Term\('content', '([^']+)'\)")
            terms = terms_pattern.findall(result_string)
            if terms:
                formatted_terms = " AND ".join(f"'{term}'" for term in terms)
        
        
        results_list.append(formatted_terms)
        #print(results)
final_df = pd.DataFrame({
    'publication_number': pub_number_list,
    'cpc_desc': results_list
})

results_list
final_df['final'] = final_df['publication_number'] + '\xa0' * 5 + final_df['cpc_desc']
pd.set_option('display.max_colwidth', None)
print(final_df['final'].to_string(index=False))

In [ ]:
new_final_df=final_df.rename(columns = {'cpc_desc': 'query'})
new_final_df
del new_final_df['final']
new_final_df.reset_index(drop=True, inplace=True) 
blankIndex=[''] * len(new_final_df)
new_final_df.index=blankIndex
new_final_df

In [ ]:
new_final_df.to_csv("submission.csv")
new_final_df